# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from citipy import citipy
# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City_ID.2,City_ID.1,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
import pandas as pd
import plotly.express as px

# Assuming city_data_df is already defined and contains 'City', 'Lat', 'Lng', and 'Humidity' columns


# Create the map
fig = px.scatter_mapbox(city_data_df, 
                        lat="Lat", 
                        lon="Lng", 
                        hover_name="City", 
                        size="Humidity",
                        color="City",
                        size_max=15,  # Adjust this value to change the maximum point size
                        zoom=3,  # Adjust the initial zoom level
                        mapbox_style="open-street-map")

# Update the layout
fig.update_layout(title="City Humidity Map",
                  height=600,
                  width = 700,
   
                  margin={"r":0,"t":0,"l":0,"b":0})

# Display the map
fig.show()

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [2]:
import pandas as pd
city_data_df = pd.read_csv("output_data/cities.csv")


# Apply criteria to filter the DataFrame
criteria_df = city_data_df[(city_data_df['Max Temp']<= 300.15) &  
                           (city_data_df['Max Temp']>= 294.15) &
                           (city_data_df['Wind Speed'] <= 4.5) &
                           (city_data_df['Wind Speed'] >= 1)
]

# Drop any rows with null values
criteria_df = criteria_df.dropna()

# Display sample data
criteria_df.count()

City_ID       0
City_ID.2     0
City_ID.1     0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       0
Date          0
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
import pandas as pd

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''  # To store hotel names retrieved from Geoapify API

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,


In [7]:


import pandas as pd

# Sample data for cities, countries, coordinates, and humidity
data = {
    'city': ['New York', 'London', 'Paris', 'Tokyo', 'Sydney'],
    'country': ['United States', 'United Kingdom', 'France', 'Japan', 'Australia'],
    'coordinates': [(40.7128, -74.0060), (51.5074, -0.1278), (48.8566, 2.3522), (35.6762, 139.6503), (-33.8651, 151.2071)],
    'humidity': [68, 72, 65, 75, 60]
}

# Creating a DataFrame
city_data_df = pd.DataFrame(data)

# Copying the DataFrame to hotel_df and adding an empty column for hotel names
hotel_df = city_data_df.copy()
hotel_df['Hotel Name'] = ''

# Displaying the copied DataFrame with the added column
hotel_df

,city,country,coordinates,humidity,Hotel Name
0,New York,United States,"(40.7128, -74.006)",68,
1,London,United Kingdom,"(51.5074, -0.1278)",72,
2,Paris,France,"(48.8566, 2.3522)",65,
3,Tokyo,Japan,"(35.6762, 139.6503)",75,
4,Sydney,Australia,"(-33.8651, 151.2071)",60,


In [6]:
import hvplot.pandas
import pandas as pd
import requests
from citipy import citipy

# Set parameters to search for a hotel
radius = 10000# YOUR CODE HERE
params = { 
    'categories': 'accommodation.hotel',
    'geoapify_key':'',  # Replace with your Geoapify API key
    'limit': 20   # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = ['Lat']
    lng = ['Lng'] # get latitude, longitude from the DataFrame
   
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"# YOUR CODE HERE
    params["bias"] =  f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary

    response=requests.get(base_url, params=params)# YOUR CODE HERE

    # Convert the API response to JSON format
    name_address =response.json() # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'city']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
New York - nearest hotel: No hotel found
London - nearest hotel: No hotel found
Paris - nearest hotel: No hotel found
Tokyo - nearest hotel: No hotel found
Sydney - nearest hotel: No hotel found


,city,country,coordinates,humidity,Hotel Name
0,New York,United States,"(40.7128, -74.006)",68,No hotel found
1,London,United Kingdom,"(51.5074, -0.1278)",72,No hotel found
2,Paris,France,"(48.8566, 2.3522)",65,No hotel found
3,Tokyo,Japan,"(35.6762, 139.6503)",75,No hotel found
4,Sydney,Australia,"(-33.8651, 151.2071)",60,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display
import hvplot.pandas
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lat",
    "Lng",
    frame_width = 600,
    frame_height = 600,
    scale = 0.5,
    color = "country"
)
map_plot

:Points   [Lat,Lng]